In [ ]:
import frame_extraction
import feature_extraction
import cv2
import os
import numpy as np

In [ ]:
from pytube import YouTube
video_url = "https://www.youtube.com/watch?v=zdkBo-GSqpo"
yt = YouTube(video_url)
stream = yt.streams.get_highest_resolution()
stream.download(filename='input_video.mp4')

'd:\\Downloads\\demo\\input_video.mp4'

In [ ]:
total_frames=frame_extraction.extract_frames('input_video.mp4',10) #path to the video, desired fps
frame_extraction.scale_down_frames('frames','train') #path of input and output folder

Number of frames extracted is  2350


In [ ]:
def load_image(path):

    image = cv2.imread(path)
    
    # Convert image in BGR format to RGB.
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Add a batch dimension which is required by the model.
    image = np.expand_dims(image, axis=0)
    
    return image

In [ ]:
#storing all the scaled down frames 
images=[]
for path in os.listdir('train'):
    images.append(load_image('train/'+path))

In [ ]:
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nlp = spacy.load("en_core_web_md")

ModuleNotFoundError: No module named 'spacy'

: 

In [ ]:
def process_query(query):

    tokens = word_tokenize(query)

    # Convert tokens to lowercase
    tokens = [word.lower() for word in tokens]

    # Remove punctuation and stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [
        word for word in tokens if word not in string.punctuation and word not in stop_words]

    return tokens

In [ ]:
def relevance_score(list1, list2):
    # Process the lists using spaCy to get Doc objects
    doc1 = nlp(" ".join(list1))
    doc2 = nlp(" ".join(list2))

    # Calculate cosine similarity between the two Doc objects
    similarity = doc1.similarity(doc2)

    return similarity

In [ ]:
feature_list={}
count=1
for x in images:
    fr=f'{count:04d}.jpg'
    fx=feature_extraction.extract_features(x,0.5)
    feature_list[fr]=fx
    print(f"Progress: {(count + 1) / total_frames * 100:.2f}%", end="\r")
    count+=1

0.42517917332802835

In [ ]:
import pickle
with open("feature_list.pkl", "wb") as f:
    pickle.dump(feature_list, f)

In [ ]:
import pickle
with open("feature_list.pkl", "rb") as f:
    feature_list = pickle.load(f)

In [ ]:
query='Car chase scene'
query=process_query(query)

selected_frames=[]
for fr,fx in feature_list.items():
    if relevance_score(query,fx)>0.3:
        selected_frames.append(fr)

In [ ]:
frame = cv2.imread('frames/0001.jpg')
height, width, layers = frame.shape

video = cv2.VideoWriter('output.mp4', 0, 4, (width,height))

for img in selected_frames:
    video.write(cv2.imread('frames/'+img))

cv2.destroyAllWindows()
video.release()